In [1]:
# !pip install transformers
# !pip install accelerate

import os
os.environ["CUDA_VISIBLE_DEVICES"]="3"

In [2]:
# #양자화에 필요한 패키지 설치
# !pip install -q -U bitsandbytes
# !pip install -q -U git+https://github.com/huggingface/transformers.git
# !pip install -q -U git+https://github.com/huggingface/peft.git
# !pip install -q -U git+https://github.com/huggingface/accelerate.git
# !pip install git+https://github.com/huggingface/peft


In [3]:
import pandas as pd
import numpy as np
import torch
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast, AdamW, AutoTokenizer
from torch.optim.lr_scheduler import CosineAnnealingLR

from tqdm import tqdm
from torch import nn
from transformers import Trainer
import torch.nn.functional as F
import torchvision.transforms as tr # 이미지 전처리 기능들을 제공하는 라이브러리
from torch.utils.data import DataLoader, Dataset # 데이터를 모델에 사용할 수 있도록 정리해 주는 라이브러리

# CUDA 사용 가능 여부 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.bfloat16
# )

In [5]:
CFG = {
    'LR' : 5e-6, # Learning Rate
    'EPOCHS' : 50, # 학습 Epoch
    'BATCH_SIZE' : 1,
    'AUG_RATIO' : 0.15,
    'AUG_PROB' : 0.5,
}

In [6]:
data_df = pd.read_csv('./data/train.csv')
# data_df = data_df.drop(['id','category'], axis=1)


In [7]:
import json
# Open the JSON file
with open('./rag_data.json', 'r') as json_file:
    # Load the JSON data
    reply_data = json.load(json_file)


list_r_6 = []
list_r_7 = []
for idx, row in reply_data.items():

    list_r_6.append(row[0].replace('\n',' '))
    list_r_7.append(row[1].replace('\n',' '))

data_df['답변_6'] = list_r_6
data_df['답변_7'] = list_r_7



In [8]:
data_df = data_df.drop(['id','category'], axis=1)
data_df

,질문_1,질문_2,답변_1,답변_2,답변_3,답변_4,답변_5,답변_6,답변_7
0,면진장치가 뭐야?,면진장치에 사용되는 주요 기술은 무엇인가요?,면진장치란 지반에서 오는 진동 에너지를 흡수하여 건물에 주는 진동을 줄여주는 진동 ...,"면진장치란 건물의 지반에서 발생하는 진동 에너지를 흡수하여 건물을 보호하고, 진동을...",면진장치란 지반으로부터 발생하는 진동 에너지를 흡수하여 건물에 전달되는 진동을 줄여...,면진장치는 건물의 지반으로부터 오는 진동 에너지를 흡수하여 건물에 전달되는 진동을 ...,면진장치는 건물에 오는 지반 진동의 영향을 최대한으로 흡수하여 건물에 전달되는 진동...,면진장치는 건축 구조물에서 수직 및 수평으로 나란히 놓여진 구조물을 연결하거나 보강...,면진장치는 건축물의 안전성을 평가하고 지지력을 유지하기 위한 중요한 요소입니다. 주...
1,내진설계의 종류 좀 알려줘,내진설계에는 어떤 종류가 있는지 자세히 알려주실 수 있나요?,"내진 설계의 종류로 내진구조, 제진구조, 면진구조가 있습니다.","내진설계에는 내진구조, 제진구조, 면진구조가 있습니다. 내진구조는 건물 구조물이 지...","내진설계에는 주로 내진구조, 제진구조, 면진구조의 세 가지 종류가 있습니다. 이들은...","내진설계에는 주로 내진구조, 제진구조, 면진구조가 사용됩니다. 내진구조는 건물 구조...","내진 설계에는 다양한 종류가 있지만, 대표적으로 내진구조, 제진구조, 면진구조가 있...",내진설계는 건물이 지진 발생 시 안전하게 유지될 수 있도록 설계하는 것을 말합니다....,"네, 지진에 대비한 건축물의 설계를 의미하는 '내진설계'에는 여러 가지 종류가 있습..."
2,철골구조의 장점이 뭐야?,철골구조의 장점을 알려줘?,철골구조는 건물의 외벽에는 그다지 하중이 걸리지 않기 때문에 고층 건물의 건축이 가...,철골구조의 장점은 건물의 외벽에는 그다지 하중이 걸리지 않기 때문에 고층 건물의 건...,철골구조의 장점은 건물의 외벽에 하중이 적게 걸리기 때문에 고층 건물의 건축이 용이...,"철골구조의 장점은 건물의 외벽이 하중이 걸리지 않아 공간 활용이 용이하고, 고층 건...",철골구조의 장점은 건물의 외벽에 하중이 크게 걸리지 않아 고층 건물을 건축할 수 있...,철골구조의 장점은 다음과 같습니다: 1. 강도: 철골은 강철로 만들어져 있어서 뛰...,철골구조는 건물을 지탱하고 지지하는 데 사용되는 구조물의 한 유형입니다. 이러한 구...
3,철골철근 콘크리트 구조가 뭐야?,철골철근 콘크리트 구조의 장점과 단점에는 무엇이 있을까요?,"철근철골콘크리트는 철골과 철근, 그리고 콘크리트를 함께 사용하는 건축 구조입니다. ...","철골철근콘크리트 구조는 건축물을 지탱하는 주요 구조물인 철골과 철근, 그리고 콘크리...",철골철근 콘크리트 구조는 건축물을 지탱하기 위한 구조물에서 일반적으로 사용되는 방식...,"철골철근콘크리트 구조는 철골과 철근, 그리고 콘크리트를 함께 사용하여 만들어지는 건...","철골철근 콘크리트 구조는 강철 골조와 강철 철근, 그리고 콘크리트를 함께 사용하여 ...",철골철근 콘크리트 구조는 건축물을 지탱하고 지지하는데 사용되는 구조적 시스템입니다....,철골철근 콘크리트 구조의 장단점은 다음과 같습니다. 장점: 1. 강도와 내구성이 ...
4,철골구조는 어떤 방식이 있어?,철골구조의 다양한 방식이 무엇인가요?,철골구조는 일반철골구조와 경량철골구조가 있습니다.,철골구조는 일반철골구조와 경량철골구조가 있습니다. 일반철골구조는 주로 대형 건물이나...,철골구조는 주로 일반철골구조와 경량철골구조로 나뉘어집니다. 이들은 건축 시스템에 따...,철골구조는 주로 일반철골구조와 경량철골구조로 구분됩니다. 이외에도 최근에는 고층 건...,철골구조는 일반철골구조와 경량철골구조 두 가지 방식이 주로 사용됩니다. 일반철골구조...,철골구조는 주로 강철을 사용하여 건물 또는 다른 구조물을 지지하는 구조 방식입니다....,"철골구조는 강철로 만들어진 구조물을 지칭하는 용어로, 다양한 방식으로 설계될 수 있..."
...,...,...,...,...,...,...,...,...,...
639,벽장 부위 결로의 원인이 뭐야?,벽장 부위 결로가 발생하는 주된 원인은 무엇일까요?,벽장 부위 결로의 원인은 난방이 이웃한 방과동일한 조건이 되나 그 방에 비해 저온인...,"벽장 부위 결로의 원인은 주로 난방이 자리잡은 방이 내부 온도가 낮은 반면, 외부 ...",벽장 부위 결로가 발생하는 원인은 난방이 잘 이루어지지 않아 해당 공간이 저온인 반...,"벽장 부위 결로가 발생하는 원인은 난방이 인접한 방과 같은 조건을 갖추고 있지만, ...",벽장 부위 결로의 주된 원인은 충분한 환기가 이루어지지 않는 환경과 과도한 습기가 ...,벽장 부위에 생긴 결로의 주요 원인은 다음과 같을 수 있습니다: 1. 고습한 환경...,벽장 부위 결로가 발생하는 주된 원인은 다음과 같습니다: 1. 습기: 과도한 습기...
640,"AD, PD에 면한 벽체 결로의 원인이 뭐야?",벽체 결로가 AD나 PD에 면한다면 그 원인이 무엇인가요?,"AD, PD에 면한 벽체 결로의 원인은 외기에 접하는 면 좌, 우측 벽체에는 단열재...","AD, PD에 면한 벽체 결로의 원인은 외기에 접하는 면 좌, 우측 벽체에는 단열재...","AD, PD에 면한 벽체 결로의 주된 원인은 외기에 접하는 면 좌, 우측 벽체에 단...","AD, PD에 면한 벽체 결로의 원인으로는 외기에 접하는 면 좌, 우측 벽체는 일반...","AD, PD에 면한 벽체 결로의 원인으로는 외부 공기에 노출된 벽면이 실내보다 냉각...",벽체 결로는 다양한 요인에 의해 발생할 수 있으나 주로 습기와 환경 조건이 원인이 ...,"벽체 결로가 AD나 PD에 면하는 경우, 이는 공간 내 습도와 온도 조절이 부족하여..."
641,외벽 모서리 부위에 결로가 발생하는 원인이 뭐야?,외벽 모서리 부위에 결로가 발생하는 것을 예방하는 방법이 있을까요?,외벽모서리 부위에 결로가 발생하는 원인은 높은 온도차 때문입니다. 외벽 모서리의 경...,외벽 모서리 부위에 결로가 발생하는 원인은 중요한 역할을 합니다. 외부에 비치는 세...,외벽 모서리 부위에 결로가 발생하는 원인으로는 높은 온도차가 대표적입니다. 외벽 모...,외벽 모서리 부위에 결로가 발생하는 원인은 실내와 외부의 온도차 때문입니다. 외벽 ...,외벽 모서리 부위에 결로가 발생하는 주요 원인은 높은 온도차입니다. 외벽 모서리는 ...,외벽 모서리 부위에 결로가 발생하는 주요 원인은 주로 습도가 높은 환경과 공기 흐름...,외벽 모서리 부위에 결로가 발생하는 것을 예방하기 위해서는 몇 가지 조치를 취할 수...
642,창호 결로의 대책은 뭐야?,창호 결로를 해결하기 위한 가장 효과적인 방법은 무엇인가요?,창호결로의 대책은 제품을 선정할 때 KS에 규정된 프레임을 선정하고 유리의 열관류율...,창호 결로의 대책은 KS에 규정된 프레임을 선정하고 유리의 열관류율 및 결로 발생 ...,"창호 결로를 예방하기 위한 대책은 KS에 규정된 프레임을 선택하고, 열관류율이 적은...","창호 결로의 대책은 여러 가지가 있습니다. 먼저, 창호를 선택할 때 KS에 규정된 ...","창호 결로를 방지하기 위한 대책으로는 KS에 규정된 프레임을 사용하고, 열관류율 및...","창호 결로는 주로 습기가 많은 환경에서 발생하는 문제로, 실내 환경의 습도를 조절하...",창호 결로는 습기와 곰팡이 성장을 촉진할 수 있는 심각한 문제입니다. 이를 해결하기...


In [9]:
# Load model directly
# from transformers import AutoTokenizer, AutoModelForCausalLM

# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM
# tokenizer = PreTrainedTokenizerFast.from_pretrained('skt/kogpt2-base-v2', eos_token='</s>')
# tokenizer = PreTrainedTokenizerFast.from_pretrained('./skt_aug2_2_29_epoch/', max_length=10000, eos_token='</s>',  truncation=True)
# model = GPT2LMHeadModel.from_pretrained("./skt_aug2_2_29_epoch/")

# tokenizer = AutoTokenizer.from_pretrained("Edentns/DataVortexS-10.7B-dpo-v1.0", eos_token='</s>', max_length=5000, truncation=True)
# model = AutoModelForCausalLM.from_pretrained("Edentns/DataVortexS-10.7B-dpo-v1.0")
model = AutoModelForCausalLM.from_pretrained("./kko_aug2_5_epoch/")
tokenizer = AutoTokenizer.from_pretrained("./kko_aug2_5_epoch/", eos_token='</s>',max_length=10000)


Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [10]:
# loaded_model = PeftModel.from_pretrained(
#     foundation_model,    # 프롬프트 튜닝에 사용될 기본 모델
#     peft_model_path,     # Peft 모델이 저장된 위치
#     is_trainable = False # 불러온 모델은 훈련될 필요 없음
# )
from peft import get_peft_model, PromptTuningConfig, TaskType, PromptTuningInit, PeftModel, PeftConfig

model = PeftModel.from_pretrained(
    model,    # 프롬프트 튜닝에 사용될 기본 모델
    "./kko_lora_7_0_epoch/",     # Peft 모델이 저장된 위치
    is_trainable = True 
)




ValueError: Can't find 'adapter_config.json' at './kko_lora_7_0_epoch/'

In [ ]:
# from transformers import AutoModelForSeq2SeqLM+ 
# from peft import PeftModel, PeftConfig 
# peft_model_id = "smangrul/twitter_complaints_bigscience_T0_3B_LORA_SEQ_2_SEQ_LM"+ config = PeftConfig.from_pretrained(peft_model_id) 
# model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path)+ model = PeftModel.from_pretrained(model, peft_model_id)
# tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
# model = model.to(device)  
# model.eval() 
# inputs = tokenizer("Tweet text : @HondaCustSvc Your customer service has been horrible during the recall process. I will never purchase a Honda again. Label :", return_tensors="pt")   
# with torch.no_grad():     
#     outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=10)  
#     print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0])  'complaint'


In [ ]:
# from peft import LoraConfig, TaskType, get_peft_model, prepare_model_for_int8_training
# lora_config = LoraConfig(r=8,
#                          lora_alpha=32,
#                          target_modules=['q_proj','v_proj'],
#                          lora_dropout=0.1,
#                          # inference_mode=False,
#                          bias="none",
#                          task_type=TaskType.CAUSAL_LM)

# model = prepare_model_for_int8_training(model)
# model = get_peft_model(model, lora_config)


In [ ]:
model.print_trainable_parameters()
# model.to(device)

In [ ]:
model

In [ ]:
import random


class HansolDataset(Dataset):
    def __init__(self, data_df, tokenizer):
        self.tokenizer = tokenizer
        self.data_df = None
        self.sentence_list = []
        self._augment_question(data_df)
        self._generate_data()
        
    def __getitem__(self, index):
        return self.sentence_list[index]
    
    def __len__(self):
        return len(self.sentence_list)

    def _augment_question(self, data_df):
        for _, row in data_df.iterrows():
            new_row = {}
            random_number = random.choice(range(0, len(data_df)))
            new_row['질문_1'] = row['질문_1'] + ' ' + data_df['질문_1'][random_number]
            new_row['질문_2'] = row['질문_2'] + ' ' + data_df['질문_2'][random_number]
            new_row['답변_1'] = row['답변_1'] + ' ' + data_df['답변_1'][random_number]
            new_row['답변_2'] = row['답변_2'] + ' ' + data_df['답변_2'][random_number]
            new_row['답변_3'] = row['답변_3'] + ' ' + data_df['답변_3'][random_number]
            new_row['답변_4'] = row['답변_4'] + ' ' + data_df['답변_4'][random_number]
            new_row['답변_5'] = row['답변_5'] + ' ' + data_df['답변_5'][random_number]
            new_row['답변_6'] = row['답변_6'] + ' ' + data_df['답변_6'][random_number]
            new_row['답변_7'] = row['답변_7'] + ' ' + data_df['답변_7'][random_number]
            df = pd.DataFrame(new_row,  index = [0])
            data_df = pd.concat([data_df, df], ignore_index=True)
        self.data_df = data_df.copy(deep=True)
        
    def _generate_data(self):
        for _, row in self.data_df.iterrows():
            for q_col in ['질문_1', '질문_2']:
                for a_col in ['답변_1', '답변_2', '답변_3', '답변_4', '답변_5', '답변_6', '답변_7']:
                    # 질문과 답변 쌍을 </s> token으로 연결
                    input_text = row[q_col] + tokenizer.eos_token + row[a_col] + tokenizer.eos_token
                    input_ids = tokenizer.encode(input_text, return_tensors='pt')
                    self.sentence_list.append(input_ids)
                    


In [ ]:

model.to(device) # 모델을 GPU단으로 이동

# 모델 학습 설정
optimizer = AdamW(model.parameters(), lr=CFG['LR'])
scheduler = CosineAnnealingLR(optimizer, T_max=100, eta_min=0.001)

model.train()

# 모델 학습
for epoch in range(CFG['EPOCHS']):
    total_loss = 0
    dataset = HansolDataset(data_df = data_df, tokenizer=tokenizer)
    dataloader = DataLoader(dataset, batch_size=CFG['BATCH_SIZE'], shuffle=True) # 미니 배치 형태로 데이터 갖추기
    progress_bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for batch_idx, batch in progress_bar:
        # 데이터를 GPU단으로 이동
        batch = batch[0]
        # batch.unsqueeze(0)
        batch = batch.to(device)
        outputs = model(batch, labels=batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        total_loss += loss.item()

        # 진행률 표시줄에 평균 손실 업데이트
        progress_bar.set_description(f"Epoch {epoch+1} - Avg Loss: {total_loss / (batch_idx+1):.4f}")
    scheduler.step()
    # 에폭의 평균 손실을 출력
    print(f"Epoch {epoch+1}/{CFG['EPOCHS']}, Average Loss: {total_loss / len(dataloader)}")
    model.save_pretrained(f"./kko_lora_8_{epoch}_epoch")
    tokenizer.save_pretrained(f"./kko_lora_8_{epoch}_epoch")